In [0]:
# Databricks Notebook: Blob to Blob Pipeline
# Purpose: Read CSVs from raw container, clean, write to processed & curated containers

from pyspark.sql.functions import current_timestamp

# -------------------------------
# Step 1: Configuration
# -------------------------------
storage_account = "<STORAGE_ACCOUNT_NAME>"
account_key = "<STOREAGE_ACCOUNT_KEY>"
#container_raw = "raw"
#container_copied = "copied"
#container_processed = "processed"
#container_curated = "curated"





In [0]:
"""
for name in ["raw", "copied", "curated"]:
    dbutils.fs.mount(
        source = f"wasbs://{name}@{storage_account}.blob.core.windows.net/",
        mount_point = f"/mnt/{name}",
        extra_configs = {
            f"fs.azure.account.key.{storage_account}.blob.core.windows.net": account_key
        }
    )
"""

In [0]:
# Set Spark config for Blob Storage access
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
    account_key
)

# -------------------------------
# Step 2: Read all CSVs from copied container
# -------------------------------
df_copied = spark.read.csv(
    "/mnt/copied/*.csv",
    header=True,
    inferSchema=True
)

print("Copied Data preview:")
display(df_copied.limit(5))

# -------------------------------
# Step 3: Simple transformation
# -------------------------------
# Drop rows with nulls and add ingestion timestamp
df_clean = df_copied.dropna().withColumn("ingestion_date", current_timestamp())

print("Cleaned data preview:")
display(df_clean.limit(5))

# -------------------------------
# Step 4: Write to processed folder
# -------------------------------
df_clean.write.mode("overwrite").csv(
    "/mnt/processed/",
    header=True
)

# -------------------------------
# Step 5: Write to curated folder
# -------------------------------
#df_clean.write.mode("append").csv(
#    f"wasbs://{container_curated}@{storage_account}.blob.core.windows.net",
#    header=True
#)

print("Data successfully written to processed folder.")
